## Imports

In [2]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usatlas/atlas01/atlasdisk/users/atlas_wifeng/sw/miniconda/envs/top-tagging/lib/python3.10/enum.py


In [4]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2023-08-23 02:40:59.978988: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 02:41:01.841180: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Loading data

defines `_load_data(particle, datafile)`

`particle` is an int to be used as the label. `datafile` is the path to a `.h5` file.

In [27]:
def _load_data(particle, datafile):
    print('load_data from datafile', datafile)
    d = h5py.File(datafile, 'r')
    first = np.expand_dims((d['layer_0'])[:], -1)
    second = np.expand_dims((d['layer_1'])[:], -1)
    third = np.expand_dims((d['layer_2'])[:], -1)
    four = np.expand_dims((d['layer_3'])[:], -1)
    energy = (d['energy'])[:].reshape(-1, 1) * 1000  # convert to MeV
    sizes = [
        first.shape[1],
        first.shape[2],
        second.shape[1],
        second.shape[2],
        third.shape[1],
        third.shape[2],
        four.shape[1],
        four.shape[2],
    ]
    y = [particle] * first.shape[0]

    return (
        first,
        second,
        third,
        four,
        y,
        energy,
        sizes,
    )

In [32]:
import yaml
with open("./config.yaml") as fin:
    data_dir = yaml.safe_load(fin)["data_dir"]
os.chdir(data_dir)

s = [
    'h5/axion1_40-250GeV_100k.h5',
    'h5/gamma_40-250GeV_100k.h5',
    'h5/pi0_40-250GeV_100k.h5'
]
events = [1000, 1000, 1000]

In [33]:
(
    first,
    second,
    third,
    four,
    y,
    energy,
    sizes,
) = [np.concatenate(t) for t in [a for a in zip(*[_load_data(1000, file) for file in s])]]

load_data from datafile h5/axion1_40-250GeV_100k.h5
load_data from datafile h5/gamma_40-250GeV_100k.h5
load_data from datafile h5/pi0_40-250GeV_100k.h5


### Load data from each layer

In [34]:
(
    first_sig,
    second_sig,
    third_sig,
    four_sig,
    y_sig,
    energy_sig,
    sizes_sig,
) = [np.concatenate(t) for t in [a for a in
     zip(*[_load_data(events[0], s[0])])]]

load_data from datafile h5/axion1_40-250GeV_100k.h5


In [35]:
(
    first_bkg0,
    second_bkg0,
    third_bkg0,
    four_bkg0,
    y_bkg0,
    energy_bkg0,
    sizes_bkg0,
) = [np.concatenate(t) for t in [a for a in
     zip(*[_load_data(events[1], s[1])])]]

load_data from datafile h5/gamma_40-250GeV_100k.h5


In [36]:
(
    first_bkg1,
    second_bkg1,
    third_bkg1,
    four_bkg1,
    y_bkg1,
    energy_bkg1,
    sizes_bkg1,
) = [np.concatenate(t) for t in [a for a in
     zip(*[_load_data(events[2], s[2])])]]

load_data from datafile h5/pi0_40-250GeV_100k.h5


### Condense, evaluate the data we have

In [37]:
(first_sig, second_sig, third_sig, four_sig, energy_sig) = \
    [(X.astype(np.float32) / 1000)[:100000] for X in [first_sig,
     second_sig, third_sig, four_sig, energy_sig]]
(first_bkg0, second_bkg0, third_bkg0, four_bkg0, energy_bkg0) = \
    [(X.astype(np.float32) / 1000)[:100000] for X in [first_bkg0,
     second_bkg0, third_bkg0, four_bkg0, energy_bkg0]]
(first_bkg1, second_bkg1, third_bkg1, four_bkg1, energy_bkg1) = \
    [(X.astype(np.float32) / 1000)[:100000] for X in [first_bkg1,
     second_bkg1, third_bkg1, four_bkg1, energy_bkg1]]

In [38]:
print ('first', first.shape)
print ('first_sig', first_sig.shape)
print ('second_sig', second_sig.shape)

first (300000, 4, 16, 1)
first_sig (100000, 4, 16, 1)
second_sig (100000, 4, 128, 1)


In [39]:
inputs = [(first, second)]  # Input Simulation
labels = np.concatenate((np.ones(first_sig.shape[0]),
                        np.zeros(first_bkg0.shape[0]),
                        np.ones(first_bkg1.shape[0]) + 1))
print(labels.shape)

(300000,)


## Build the model

In [5]:
print("hello")

hello


In [20]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


In [21]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [22]:
merge = concatenate([flat1, flat2])

In [23]:
# interpretation model
hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(10, activation='relu')(hidden1)
output = Dense(3, activation='sigmoid')(hidden2)
cnn = Model(inputs=[visible1, visible2], outputs=output)

cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [24]:
# summarize layers
print(cnn.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 4, 16, 1)]   0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 4, 128, 1)]  0           []                               
                                                                                                  
 conv2d_13 (Conv2D)             (None, 4, 16, 32)    544         ['input_9[0][0]']                
                                                                                                  
 conv2d_15 (Conv2D)             (None, 4, 128, 32)   544         ['input_10[0][0]']               
                                                                                            

## Train the model

In [40]:
history = cnn.fit(inputs, labels, epochs=50, batch_size=100)

Epoch 1/50


2023-08-23 02:46:44.518957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-08-23 02:46:52.567459: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2ad85ae1a6f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-23 02:46:52.567499: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2023-08-23 02:46:52.573609: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-23 02:46:52.806416: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3000/3000 [==============================] - 27s 4ms/step - loss: 0.9821 - acc: 0.4543
Epoch 2/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.4095 - acc: 0.8398
Epoch 3/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.1552 - acc: 0.9551
Epoch 4/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.1066 - acc: 0.9674
Epoch 5/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.0897 - acc: 0.9724
Epoch 6/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.0821 - acc: 0.9747
Epoch 7/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.0759 - acc: 0.9765
Epoch 8/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.0702 - acc: 0.9785
Epoch 9/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.0672 - acc: 0.9794
Epoch 10/50
3000/3000 [==============================] - 12s 4ms/step - loss: 0.0638 - acc: 0.9805
Epoch 11/50
3000/3000 [=======